In [ ]:
from jupyter_compare_view import compare
import ipywidgets as widgets

import copy
import numpy as np

from scantools import logger
from scantools.capture import Capture

from scantools.proc.rendering import Renderer
from scantools.utils.io import read_mesh, read_image

In [ ]:
capture_path = "/home/plukovic/research_assistant/capture/HYDRO"
query_id = "ios_2023-10-27_11.53.29_000"
ref_id = "2023-11-03_10.31.58+2023-11-03_13.51.06"
skip = 10

In [ ]:
capture = Capture.load(capture_path)

In [ ]:
def read_raw_image(cam_id, data_path, images):
    if cam_id not in images:
        # It's a rig. Pick first camera.
        cam_id = list(sorted(images.keys()))[0]
    return read_image(data_path / images[cam_id])


def render_image(cam_id, T, images, cameras, renderer, rig=None):
    T = copy.deepcopy(T)
    if cam_id not in images:
        # It's a rig. Pick first camera.
        cam_id = list(sorted(images.keys()))[0]
        T_cam2rig = rig[cam_id]
        T = T * T_cam2rig
    camera = cameras[cam_id]
    render, _ = renderer.render_from_capture(T, camera)
    render = (np.clip(render, 0, 1) * 255).astype(np.uint8)
    return render

In [ ]:
class ButtonHandler:
    def __init__(self, trajectory, session, data_path, renderer, skip=5):
        self.idx = 0
        self.render = False
        self.keys = list(sorted(trajectory.key_pairs()))
        self.trajectory = trajectory
        self.session = session
        self.data_path = data_path
        self.renderer = renderer
        self.skip = skip
        self.ts, self.cam_id = sorted(self.trajectory.key_pairs())[0]
        self.raw_img = read_raw_image(self.cam_id, capture.data_path(query_id), session_q.images[self.ts])
        self.render_img = render_image(self.cam_id, self.trajectory[self.ts, self.cam_id], self.session.images[self.ts], self.session.sensors, self.renderer, rig=(self.session.rigs[self.cam_id] if self.cam_id in self.session.rigs else None))
    
    # This function is called when bprev is clicked
    def previous(self, event):
        self.idx = (self.idx - self.skip + len(self.keys)) % len(self.keys)
        self.render = False
        ts, cam_id = self.keys[self.idx]
        self.raw_img = read_raw_image(cam_id, self.data_path, self.session.images[ts])
        self.render_img = render_image(cam_id, self.trajectory[ts, cam_id], self.session.images[ts], self.session.sensors, self.renderer, rig=(self.session.rigs[cam_id] if cam_id in self.session.rigs else None))
        
    # This function is called when bnext is clicked
    def next(self, event):
        self.idx = (self.idx + self.skip) % len(self.keys)
        self.render = False
        ts, cam_id = self.keys[self.idx]
        self.raw_img = read_raw_image(cam_id, self.data_path, self.session.images[ts])
        self.render_img = render_image(cam_id, self.trajectory[ts, cam_id], self.session.images[ts], self.session.sensors, self.renderer, rig=(self.session.rigs[cam_id] if cam_id in self.session.rigs else None))

In [ ]:
session_ref = capture.sessions[ref_id]
T_mesh2global = session_ref.proc.alignment_global.get_abs_pose('pose_graph_optimized')
session_q = capture.sessions[query_id]

logger.info('Generating interactive visualization diffs by rendering.')
mesh = read_mesh(capture.proc_path(ref_id) / session_ref.proc.meshes['mesh'])
renderer = Renderer(mesh)
trajectory = session_q.proc.alignment_trajectories
if T_mesh2global is not None:
    trajectory = T_mesh2global.inv * trajectory

callback = ButtonHandler(trajectory, session_q, capture.data_path(query_id), renderer, skip)

bnext = widgets.Button(description='Next')
bnext.on_click(callback.next)

bprev = widgets.Button(description='Prev')
bprev.on_click(callback.previous)

widgets.VBox([bnext, bprev])

In [ ]:
compare(callback.render_img, callback.raw_img, cmap="gray", start_mode="horizontal", start_slider_pos=0.5)